# Spider Fly testing environment

In [5]:
from custom_agent.CTCE.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo_testing import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

import numpy as np

import pandas as pd

import torch

env = SpiderFlyEnvMA(max_steps = 100)
env = PettingZooWrapper(env, normalize = True)

agents = Agents(env, batch_size = 256, layer_sizes = (64, 64), global_observations = True) 
# agents = Agents(env, batch_size = 256, global_observations = True) 

In [6]:
rewards = agents.train(nr_steps = 30000)

KeyboardInterrupt: 

In [9]:
from custom_agent.CTCE.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

import numpy as np

import pandas as pd

import torch

env = SpiderFlyEnvMA(spiders=2, size=4, max_timesteps = 100)
env = PettingZooWrapper(env, normalize = True)

agents = Agents(env, batch_size = 256, layer_sizes = (64, 64), global_observations = True) 
# agents = Agents(env, batch_size = 256, global_observations = True) 

In [10]:
rewards = agents.train(nr_steps = 500 * 100)

[Episode 50 total reward: [-0.173 -0.2  ]] ~ 
[Episode 100 total reward: [2.83  2.872]] ~ 
[Episode 150 total reward: [3.835 3.964]] ~ 
[Episode 200 total reward: [10.012  9.862]] ~ 
[Episode 250 total reward: [34.96  34.924]] ~ 
[Episode 300 total reward: [28.942 28.945]] ~ 
[Episode 350 total reward: [29.911 29.986]] ~ 


KeyboardInterrupt: 

In [1]:
from custom_agent.CTCE.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

import numpy as np

import pandas as pd

env = SpiderFlyEnvMA(spiders=3, size=4, max_timesteps = 100)
env = PettingZooWrapper(env, normalize = True)

agents = Agents(env, batch_size = 256, layer_sizes = (256, 256), global_observations = True) 
# agents = Agents(env, batch_size = 256, global_observations = True) 

agents.train(nr_steps = 3000 * 100, warmup_steps = 20000)

/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGrid-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGridMA-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return 

[Episode 50 total reward: [1.816 1.807 1.831]] ~ 
[Episode 100 total reward: [-0.134 -0.158 -0.161]] ~ 
[Episode 150 total reward: [-0.104 -0.158 -0.137]] ~ 
[Episode 200 total reward: [-0.098 -0.173 -0.113]] ~ 
[Episode 250 total reward: [0.808 0.868 0.868]] ~ 
[Episode 300 total reward: [0.898 0.886 1.006]] ~ 
[Episode 350 total reward: [-0.044 -0.149 -0.2  ]] ~ 
[Episode 400 total reward: [0.865 0.931 0.967]] ~ 
[Episode 450 total reward: [0.802 1.09  1.078]] ~ 
[Episode 500 total reward: [4.012 3.892 4.036]] ~ 
[Episode 550 total reward: [-0.2   -0.002  0.1  ]] ~ 
[Episode 600 total reward: [1.918 2.065 1.957]] ~ 
[Episode 650 total reward: [4.078 4.039 3.847]] ~ 
[Episode 700 total reward: [6.973 6.997 6.877]] ~ 
[Episode 750 total reward: [4.075 3.949 3.823]] ~ 
[Episode 800 total reward: [0.82  1.075 0.958]] ~ 
[Episode 850 total reward: [5.065 4.861 4.819]] ~ 
[Episode 900 total reward: [1.09  0.802 0.988]] ~ 
[Episode 950 total reward: [8.893 9.004 8.986]] ~ 
[Episode 1000 tot

In [6]:
agents.actor.save("models/SAC_hard3", "actor")
agents.critic1.save("models/SAC_hard3", "critic1")
agents.critic2.save("models/SAC_hard3", "critic2")
agents.critic1_targ.save("models/SAC_hard3", "critic1_targ")
agents.critic1_targ.save("models/SAC_hard3", "critic2_targ")

# agents.actor.save_checkpoint("models/SAC_hard3", "actor", loss = 0, step = 1000*100)
# agents.critic1.save_checkpoint("models/SAC_hard3", "critic1", loss = 0, step = 1000*100)
# agents.critic2.save_checkpoint("models/SAC_hard3", "critic2", loss = 0, step = 1000*100)
# agents.critic1_targ.save_checkpoint("models/SAC_hard3", "critic1_targ", loss = 0, step = 1000*100)
# agents.critic1_targ.save_checkpoint("models/SAC_hard3", "critic2_targ", loss = 0, step = 1000*100)


# TEST of custom Multi-Agent SAC agent on citylearn env

In [ ]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import torch

import numpy as np
import pandas as pd

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedSpaceWrapper, StableBaselines3Wrapper

from custom_agent.CTDE.ma_sac_agents_seq2 import Agents

from custom_reward.custom_reward import CustomReward

In [ ]:
schema_path = "data/schema.json"

env = CityLearnEnv(schema = schema_path, reward_function = CustomReward, central_agent=False)

# wrap environment for a more workable env
env = NormalizedSpaceWrapper(env)

In [ ]:
sac_agent = Agents(env, batch_size=100, buffer_max_size=100000)

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
warmup_steps = 5000

# make agent
sac_agent = Agents(env, batch_size=256, buffer_max_size=100000)

# training run
sac_agent.train(nr_steps = 5500 * 720, warmup_steps = warmup_steps, learn_delay = 100, learn_freq = 1, learn_weight = 1, save_dir = "temp_models")
